# Corruption

In [1]:
import pandas as pd
import numpy as np
import bs4
from bs4 import BeautifulSoup
import urllib2

In [2]:
#download html table

url = 'https://15mpedia.org/wiki/Lista_de_casos_de_corrupci%C3%B3n'

#read url
html = urllib2.urlopen(url).read()
soup = BeautifulSoup(html)
tables = soup.findAll("table")

Caso = []
Ambito = []
Partido = []

for table in tables:
    for row in table.findAll("tr"):
        cells = row.findAll("td")
        #For each "tr", assign each "td" to a variable.
        if len(cells) == 4:
            
            Caso.append(cells[0].find(text=True).rstrip())
            
            if cells[1].find(text=True) == None: Ambito.append(np.nan)
            else: Ambito.append(cells[1].find(text=True).rstrip())
                
            if cells[2].find(text=True) == None: Partido.append(np.nan)
            else: Partido.append(cells[2].find(text=True).rstrip())
                
lista_corrupcion = pd.DataFrame()
lista_corrupcion['Caso'] = Caso
lista_corrupcion['Comunidad'] = Ambito
lista_corrupcion['Partido'] = Partido

lista_corrupcion.head()

,Caso,Comunidad,Partido
0,Caso AVE,Nacional,Partido Socialista Obrero Español
1,Caso Acuamed,Nacional,Partido Popular
2,Caso Almenas,puebla de almenara,Partido Socialista Obrero Español
3,Caso Andratx,Baleares,Partido Popular
4,Caso Arcos,Andalucía,Partido Popular


In [3]:
#get latitude and longitudes of each capital communities
import re

url = 'https://es.wikipedia.org/wiki/Anexo:Capitales_de_comunidades_aut%C3%B3nomas_de_Espa%C3%B1a'

#read url
html = urllib2.urlopen(url).read()
soup = BeautifulSoup(html)
tables = soup.findAll("table")

Comunidad = []
Capital = []
Coordenadas = []

for table in tables:
    for row in table.findAll("tr"):
        cells = row.findAll("td")
        #For each "tr", assign each "td" to a variable.
        if len(cells) == 6:
            
            Comunidad.append(re.findall('title="(.*?)">', str(cells[0]))[0].decode('utf-8'))
            Capital.append(re.findall('title="(.*?)">', str(cells[1]))[0])
            Coordenadas.append(re.findall(r"\d+\.\d+", str(cells[5]))[0:2])
            
Comunidad[3] = Comunidad[3].replace(Comunidad[3], 'Baleares')
         
ccaa_coord = pd.DataFrame()
ccaa_coord['Comunidad'] = Comunidad
ccaa_coord['Capital'] = Capital
ccaa_coord['Coordenadas'] = Coordenadas

ccaa_coord.head()

,Comunidad,Capital,Coordenadas
0,Andalucía,Sevilla,"[37.367, 5.983]"
1,Aragón,Zaragoza,"[41.650, 0.883]"
2,Asturias,Oviedo,"[43.36000, 5.84500]"
3,Baleares,Palma de Mallorca,"[39.567, 2.650]"
4,Canarias,Santa Cruz de Tenerife,"[28.150, 15.417]"


In [4]:
df = lista_corrupcion.merge(ccaa_coord,  how='left')
cols = df.columns.tolist()
# reorder columns
cols = [cols[0]] + [cols[2]] + [cols[1]] + [cols[3]] + [cols[4]]
# "commit" the reordering
df = df[cols]
df = df.dropna()
df[3:7]

,Caso,Partido,Comunidad,Capital,Coordenadas
6,Caso Arroyo,Partido Popular,Castilla y León,Valladolid,"[41.633, 4.717]"
7,Caso Astapa,Partido Socialista Obrero Español,Andalucía,Sevilla,"[37.367, 5.983]"
9,Caso Baltar,Partido Popular,Galicia,Santiago de Compostela,"[42.867, 8.550]"
10,Caso Banca Catalana,Convergència i Unió,Cataluña,Barcelona,"[41.383, 2.167]"


In [7]:
# create json file to read in javascript
import json
from pprintpp import pprint as pp

# write df as JSON, bring into appropriate format
obj = {'objects': list()}
for index, ccaa in df['Comunidad'].iteritems():
    df_ccaa = df[df['Comunidad'] == ccaa]
    obj['objects'].append({'circle': {
                'coordenadas': df_ccaa.Coordenadas.values[0], 
                'comunidad': df_ccaa.Comunidad.values[0],
                'capital': df_ccaa.Capital.values[0],
                'caso': [value for value in df_ccaa.Caso.values],
                'partido': [value for value in df_ccaa.Partido.values]
            }})

with open('../data/casos_comunidad.json', 'w') as data_file:    
    json.dump(obj, data_file)
    
pp(obj['objects'][:1])


[
    {
        'circle': {
            'capital': 'Palma de Mallorca',
            'caso': [
                u'Caso Andratx',
                u'Caso Bon Sosec',
                u'Caso Funeraria',
                u'Caso Ibatur/Pasarela',
                u'Caso Palma Arena',
                u'Caso Peaje',
                u'Caso Picnic',
                u'Caso Rasputín',
                u'Caso Túnel de Sóller',
            ],
            'comunidad': u'Baleares',
            'coordenadas': ['39.567', '2.650'],
            'partido': [
                u'Partido Popular',
                u'Partido Popular',
                u'Partido Popular',
                u'Partido Popular',
                u'Partido Popular',
                u'Unió Mallorquina',
                u'Unió Mallorquina',
                u'Partido Popular',
                u'Partido Popular',
            ],
        },
    },
]
